In [60]:
import sys
if ".." not in sys.path:
    sys.path.insert(0, "..")

In [61]:
from cobra.io import read_sbml_model

model = read_sbml_model("../model/Rpom_05.xml")

In [62]:
def get_glc_flux(rxnid):
    with model:
        model.reactions.get_by_id("EX_glc").bounds = (-5.44, 0)

        sol = model.optimize()
    return sol.fluxes[rxnid]

In [63]:
import pandas as pd

# Check for class-looking names
suspicious_names = pd.DataFrame([
    {
        "ID": met.id,
        "Reactions": ", ".join([rxn.id for rxn in met.reactions]),
        "Link 1": f"https://biocyc.org/compound?orgid=RUEGERIA_POMEROYI_DSS3&id={met.id[:-3]}",
        "Link 2": f"https://biocyc.org/compound?orgid=GCF_000011965&id={met.id[:-3]}",
        "Total Abs Flux": sum(abs(get_glc_flux(rxn.id)) for rxn in met.reactions)
    }
    for met in model.metabolites
    if (met.id[:-3].upper() != met.id[:-3]
        and met.id[:-3] != "Pi")]).sort_values("Total Abs Flux", ascending=False)

suspicious_names.to_clipboard(index=False)
suspicious_names

,ID,Reactions,Link 1,Link 2,Total Abs Flux
40,Cytochromes-C-Oxidized[p],"CYTOCHROME-C-OXIDASE-RXN, RXN-14107, HAONITRO-...",https://biocyc.org/compound?orgid=RUEGERIA_POM...,https://biocyc.org/compound?orgid=GCF_00001196...,35.523692
41,Cytochromes-C-Reduced[p],"CYTOCHROME-C-OXIDASE-RXN, RXN-14107, HAONITRO-...",https://biocyc.org/compound?orgid=RUEGERIA_POM...,https://biocyc.org/compound?orgid=GCF_00001196...,35.523692
37,Glucopyranose[c],"GLCtpp, GLUCISOM-RXN, GLUCOKIN-RXN",https://biocyc.org/compound?orgid=RUEGERIA_POM...,https://biocyc.org/compound?orgid=GCF_00001196...,10.880000
109,Glucopyranose[p],"GLCtex, GLCtpp",https://biocyc.org/compound?orgid=RUEGERIA_POM...,https://biocyc.org/compound?orgid=GCF_00001196...,10.880000
110,Glucopyranose[e],"GLCtex, EX_glc",https://biocyc.org/compound?orgid=RUEGERIA_POM...,https://biocyc.org/compound?orgid=GCF_00001196...,10.880000
...,...,...,...,...,...
59,3-hydroxy-cis-D7-tetraecenoyl-ACPs[c],"RXN-10655, RXN-10656",https://biocyc.org/compound?orgid=RUEGERIA_POM...,https://biocyc.org/compound?orgid=GCF_00001196...,0.000000
58,Cis-Delta5-dodecenoyl-ACPs[c],"RXN0-2145, RXN-10654",https://biocyc.org/compound?orgid=RUEGERIA_POM...,https://biocyc.org/compound?orgid=GCF_00001196...,0.000000
51,D-Ribofuranose[c],"RIBOKIN-RXN, RXN0-366, RXN0-363",https://biocyc.org/compound?orgid=RUEGERIA_POM...,https://biocyc.org/compound?orgid=GCF_00001196...,0.000000
46,Palmitoleoyl-ACPs[c],"RXN-9550, RXN-10661, PALMITOTRANS-RXN",https://biocyc.org/compound?orgid=RUEGERIA_POM...,https://biocyc.org/compound?orgid=GCF_00001196...,0.000000


In [64]:
instanced_reactions = pd.DataFrame([
    {
        "ID": rxn.id,
        "Stem": rxn.notes.get("stem", ""),
        "GPR": rxn.gene_reaction_rule,
        "Reaction": rxn.reaction,
        "Glucose flux": get_glc_flux(rxn.id)
    }
    for rxn in model.reactions
    if "//" in rxn.id
]).sort_values("Stem")

instanced_reactions.to_clipboard(index=False)
instanced_reactions

,ID,Stem,GPR,Reaction,Glucose flux
0,1.4.1.21-RXN-L-ASPARTATE/NAD/WATER//OXALACETIC...,1.4.1.21-RXN,SPO_RS14615,L-ASPARTATE[c] + NAD[c] + WATER[c] <=> AMMONIU...,0.000000
1,1.4.1.21-RXN-L-ASPARTATE/NADP/WATER//OXALACETI...,1.4.1.21-RXN,SPO_RS14615,L-ASPARTATE[c] + NADP[c] + WATER[c] <=> AMMONI...,0.000000
2,1.5.5.1-RXN-ETF-Reduced/UBIQUINONE-10//ETF-Oxi...,1.5.5.1-RXN,,ETF-Reduced[c] + UBIQUINONE-10[c] --> CPD-9958...,0.738584
3,2.6.1.57-RXN-PHE/2-KETOGLUTARATE//PHENYL-PYRUV...,2.6.1.57-RXN,SPO3720,2-KETOGLUTARATE[c] + PHE[c] <=> GLT[c] + PHENY...,0.000000
4,2.7.1.156-RXN-ITP/ADENOSYLCOBINAMIDE//ADENOSYL...,2.7.1.156-RXN,SPO_RS02050,ADENOSYLCOBINAMIDE[c] + ITP[c] --> ADENOSYLCOB...,0.000000
...,...,...,...,...,...
103,TRANS-RXN1G48-5-K+/ACET//K+/ACET.17.,TRANS-RXN1G48-5,,ACET[p] + K+[p] --> ACET[c] + K+[c],0.000000
104,TRANS-RXN1G48-5-ZN+2/ACET//ZN+2/ACET.21.,TRANS-RXN1G48-5,,ACET[p] + ZN+2[p] --> ACET[c] + ZN+2[c],0.000000
106,TRANS-RXN1G48-9-VAL/PROTON//VAL/PROTON.23.,TRANS-RXN1G48-9,,PROTON[p] + VAL[p] --> PROTON[c] + VAL[c],0.000000
105,TRANS-RXN1G48-9-GLT/PROTON//GLT/PROTON.23.,TRANS-RXN1G48-9,,GLT[p] + PROTON[p] --> GLT[c] + PROTON[c],0.000000


In [65]:
[met for met in model.metabolites if met.formula=="C5H10O5"]

[<Metabolite RIBOSE[e] at 0x7ad424769ee0>,
 <Metabolite D-Ribofuranose[c] at 0x7ad42463a510>,
 <Metabolite RIBOSE[p] at 0x7ad4249272c0>,
 <Metabolite D-Xylopyranose[c] at 0x7ad424603950>,
 <Metabolite RIBOSE[c] at 0x7ad4248aedb0>,
 <Metabolite D-XYLULOSE[c] at 0x7ad42488de20>,
 <Metabolite D-Xylopyranose[e] at 0x7ad424a4e420>,
 <Metabolite D-Xylopyranose[p] at 0x7ad424a4e3c0>]

In [71]:
for rxn in model.metabolites.get_by_id("RIBOSE[c]").reactions:
    print(f"{rxn.id} ({get_glc_flux(rxn.id)})\n\t{rxn.reaction}")

TRANS-RXN-332 (0.0)
	ATP[c] + RIBOSE[p] + WATER[c] --> ADP[c] + PROTON[c] + Pi[c] + RIBOSE[c]


In [74]:
model.reactions.get_by_id('TRANS-RXN-332').gene_reaction_rule

''

In [70]:
model.metabolites.get_by_id("D-Ribofuranose[c]")

Metabolite identifier,D-Ribofuranose[c]
Name,D-ribofuranose
Memory address,0x7ad42463a510
Formula,C5H10O5
Compartment,c
In 3 reaction(s),"RIBOKIN-RXN, RXN0-366, RXN0-363"


In [ ]:
with model:
    model.reactions.get_by_id("EX_glc").bounds = (-5.44, 0)

    # from cobra import Reaction
    # biotin_tpp = Reaction("BIOTINtpp", lower_bound=-1000.0, upper_bound=1000.0)
    # model.add_reactions([biotin_tpp])
    # biotin_tpp.build_reaction_from_string("BIOTIN[e] <=> BIOTIN[p]")
    
    sol = model.optimize()
sol

,fluxes,reduced_costs
1-ACYLGLYCEROL-3-P-ACYLTRANSFER-RXN,0.043226,0.000000e+00
1.1.1.178-RXN,0.000000,-3.169325e-02
1.1.1.262-RXN,0.000000,-9.006069e-17
1.1.1.39-RXN,0.000000,-1.056442e-02
1.1.1.8-RXN,0.000000,1.455718e-16
...,...,...
RXN-20676,0.000000,-1.162086e-01
RXN-15714,0.000000,-2.033650e-01
RXN-8752,0.000000,-3.697546e-02
RXN-22610,0.000000,-5.551115e-17


In [ ]:
with model:
    model.reactions.get_by_id("EX_glc").bounds = (-5.44, 0.0)

    sol = model.optimize().fluxes

glc_fluxes = pd.DataFrame([
    {
        "ID": rxn,
        "Flux" : flux,
        "Abs Flux" : abs(flux),
        "GPR": model.reactions.get_by_id(rxn).gene_reaction_rule
    }
    for _, (rxn, flux) in sol.reset_index().iterrows()
]).sort_values("Abs Flux", ascending=False)

glc_fluxes.to_clipboard(index=False)
glc_fluxes

,ID,Flux,Abs Flux,GPR
452,Htex,83.717732,83.717732,
178,ATPSYN-RXN,-32.810517,32.810517,SPO0183
1430,1.10.2.2-RXN,23.606693,23.606693,SPO0273 and SPO0272 and SPO0271
320,EX_h2o,22.480439,22.480439,
423,H2Otpp,-22.480439,22.480439,
...,...,...,...,...
780,RXN-12864,0.000000,0.000000,
779,RXN-12754,0.000000,0.000000,
778,RXN-12753,0.000000,0.000000,
777,RXN-12752,0.000000,0.000000,SPO_RS11625


In [ ]:
model.reactions.get_by_id("RXN1RHL-6")

KeyError: 'RXN1RHL-6'